In [ ]:
""" Work bench for tuning NN model."""
%load_ext tensorboard

import os
import json
from pprint import pprint
from datetime import datetime

from dotenv import load_dotenv


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, AutoTokenizer, AutoConfig, TFDistilBertModel, TFBertModel, TFTrainingArguments
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tc_data import TopCoder

load_dotenv()
pd.set_option('display.max_rows', 800)

In [ ]:
tc = TopCoder()

In [ ]:
meta_inputs = tf.keras.Input(shape=(36,), name='meta_data_input')
bert_inputs = tf.keras.Input(shape=(48,), name='pool_bert_input')

concat = tf.keras.layers.concatenate([bert_inputs, meta_inputs], name='concat')
dense = tf.keras.layers.Dense(512, activation='relu', name='pre_regression')(concat)

score_output = tf.keras.layers.Dense(4, name='score')(dense)
registration_output = tf.keras.layers.Dense(1, name='registration')(dense)
sub_reg_output = tf.keras.layers.Dense(1, name='sub_reg_ratio')(dense)

mtl_model = tf.keras.Model(inputs=[bert_inputs, meta_inputs], outputs=[score_output, registration_output, sub_reg_output])

tf.keras.utils.plot_model(mtl_model, to_file='model_arch/tmp.png', show_shapes=True)